In [1]:
# %help

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.4 


In [2]:
%stop_session

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.4 
There is no current session.


In [11]:
%additional_python_modules [awswrangler=3.7.2]
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2

You are already connected to a glueetl session 40888b3f-91ff-4a79-9d99-1a05f24cebb0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Additional python modules to be included:
awswrangler=3.7.2


You are already connected to a glueetl session 40888b3f-91ff-4a79-9d99-1a05f24cebb0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 40888b3f-91ff-4a79-9d99-1a05f24cebb0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 40888b3f-91ff-4a79-9d99-1a05f24cebb0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 40888b3f-91ff-4a79-9d99-1a05f24cebb0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 2
Setting new number of workers to: 2


####  Setup e imports.


In [10]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import pyspark.pandas as ps
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [17]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Biblioteca que trata dados desbalanceados através da técnica de oversampling
# from imblearn.over_sampling import RandomOverSampler
from collections import Counter

#### Leitura da tabela e impressão do esquema do dataframe


In [4]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='banking_tables', table_name='clientes_bancarios')
dyf.printSchema()

root
|-- rownumber: long
|-- customerid: long
|-- surname: string
|-- creditscore: long
|-- geography: string
|-- gender: string
|-- age: double
|-- tenure: long
|-- balance: double
|-- numofproducts: long
|-- hascrcard: long
|-- isactivemember: long
|-- estimatedsalary: double
|-- exited: long


#### Imprimindo amostra


In [5]:
df = dyf.toDF()
df.show()

+---------+----------+---------+-----------+---------+------+-----+------+---------+-------------+---------+--------------+---------------+------+
|rownumber|customerid|  surname|creditscore|geography|gender|  age|tenure|  balance|numofproducts|hascrcard|isactivemember|estimatedsalary|exited|
+---------+----------+---------+-----------+---------+------+-----+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42.0|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41.0|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42.0|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39.0|     1|      0.0|            2|        0|          

In [20]:
df_clientes = df.toPandas()


In [7]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10002 non-null  int64  
 1   customerid       10002 non-null  int64  
 2   surname          10002 non-null  object 
 3   creditscore      10002 non-null  int64  
 4   geography        10002 non-null  object 
 5   gender           10002 non-null  object 
 6   age              10001 non-null  float64
 7   tenure           10002 non-null  int64  
 8   balance          10002 non-null  float64
 9   numofproducts    10002 non-null  int64  
 10  hascrcard        10001 non-null  float64
 11  isactivemember   10001 non-null  float64
 12  estimatedsalary  10002 non-null  float64
 13  exited           10002 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 1.1+ MB


In [8]:
# Verificando valores nulos
df_clientes.isnull().sum()

rownumber          0
customerid         0
surname            0
creditscore        0
geography          0
gender             0
age                1
tenure             0
balance            0
numofproducts      0
hascrcard          1
isactivemember     1
estimatedsalary    0
exited             0
dtype: int64


In [9]:
# Verificando valores duplicados
df_clientes.duplicated().sum()

2


In [10]:
#quantidade atual de registros
len(df_clientes)

10002


In [28]:
df_clientes_saida = df_clientes.drop(['customerid', 'surname', 'rownumber'], axis = 1)
df_clientes_saida

       creditscore geography  gender  ...  isactivemember  estimatedsalary  exited
0              619    France  Female  ...             1.0        101348.88       1
1              608     Spain  Female  ...             1.0        112542.58       0
2              502    France  Female  ...             0.0        113931.57       1
3              699    France  Female  ...             0.0         93826.63       0
4              850     Spain  Female  ...             1.0         79084.10       0
...            ...       ...     ...  ...             ...              ...     ...
9997           709    France  Female  ...             1.0         42085.58       1
9998           772   Germany    Male  ...             0.0         92888.52       1
9999           772   Germany    Male  ...             0.0         92888.52       1
10000          792    France  Female  ...             0.0         38190.78       0
10001          792    France  Female  ...             0.0         38190.78       0

[10

In [29]:
#Exclusão de duplicados
df_clientes_saida = df_clientes_saida.drop_duplicates()

In [30]:
#Exclusão de Nulos
df_clientes_saida = df_clientes_saida.dropna()

In [31]:
len(df_clientes_saida)

9997


In [32]:
# Codificando one-hot para as colunas 'Geography' e 'Gender'
df_clientes_saida = pd.get_dummies(data=df_clientes_saida, columns=['geography', 'gender'], dtype=bool)
df_clientes_saida.head()

   creditscore   age  tenure  ...  geography_Spain  gender_Female  gender_Male
0          619  42.0       2  ...            False           True        False
1          608  41.0       1  ...             True           True        False
2          502  42.0       8  ...            False           True        False
3          699  39.0       1  ...            False           True        False
5          645  44.0       8  ...             True          False         True

[5 rows x 15 columns]


#### Escrita da tabela modificada na camada curated


In [33]:
df_clientes_saida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9997 entries, 0 to 10000
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   creditscore        9997 non-null   int64  
 1   age                9997 non-null   float64
 2   tenure             9997 non-null   int64  
 3   balance            9997 non-null   float64
 4   numofproducts      9997 non-null   int64  
 5   hascrcard          9997 non-null   float64
 6   isactivemember     9997 non-null   float64
 7   estimatedsalary    9997 non-null   float64
 8   exited             9997 non-null   int64  
 9   geography_         9997 non-null   bool   
 10  geography_France   9997 non-null   bool   
 11  geography_Germany  9997 non-null   bool   
 12  geography_Spain    9997 non-null   bool   
 13  gender_Female      9997 non-null   bool   
 14  gender_Male        9997 non-null   bool   
dtypes: bool(6), float64(5), int64(4)
memory usage: 839.6 KB


In [40]:
#transformacao de tipagem dos dataframes para o padrão de saída do glue job
psser = ps.from_pandas(df_clientes_saida)

pssdf = psser.to_spark()

df_saida_glue = DynamicFrame.fromDF(pssdf, glueContext, "df_saida_glue")

In [45]:
s3_path = "s3://grupo3-2024-1-machine-learning/curated/"
s3output = glueContext.purge_s3_path(s3_path,  {"retentionPeriod": 0})
s3output = glueContext.getSink(
  path=s3_path,
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
mode = "overwrite"
)
s3output.setCatalogInfo(
  catalogDatabase="banking_tables", catalogTableName="clientes_bancarios_limpa"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(df_saida_glue)